In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

### Word Embeddings

In [12]:
word_to_ix = {'hello': 0, 'world': 1}
embeds = nn.Embedding(2, 5) # 2 words in vocabulary, 5 dimensional embeddings
print(embeds)
lookup_tensor = torch.tensor([word_to_ix['hello']], dtype=torch.long)
lookup_tensor2 = torch.tensor([word_to_ix['world']], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
world_embed = embeds(lookup_tensor2)
print(hello_embed)
print(world_embed)

Embedding(2, 5)
tensor([[-0.6101, -1.2960, -0.9434,  0.6684,  1.1628]])
tensor([[-0.3229,  1.8782, -0.5666,  0.4016, -0.1153]])


### N-Gram Language Modeling

In [17]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

# 词嵌入(word embedding)，3个词一组，前两个是context，第三个是target
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2]) for i in range(len(test_sentence) - 2)]
print(trigrams[:3])

vocab = set(test_sentence) # 词汇表，去重后看有哪些单词
print('vocab size: ', len(vocab))

word_to_ix = {word: i for i, word in enumerate(vocab)} # 构建词字典，word: index

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
vocab size:  97


In [18]:
class NGramLanguageModeler(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

### 计算词嵌入的Loss并优化

In [23]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:
        
        # step 1: 将words标记为integer indices并包裹在tensor中
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        
        # step 2: Recall that torch *accumulates* gradients.
        # Before passing in a new instance, you need to zero
        # out the gradients from the old instance
        model.zero_grad()
        
        # Step 3: Run the forward pass, getting log probabilities
        # over next words
        log_probs = model(context_idxs)
        
        # step 4
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
        
        # step 5
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)

[tensor([ 521.6913]), tensor([ 519.1707]), tensor([ 516.6678]), tensor([ 514.1837]), tensor([ 511.7177]), tensor([ 509.2679]), tensor([ 506.8328]), tensor([ 504.4120]), tensor([ 502.0040]), tensor([ 499.6068])]


In [4]:
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1], raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
    
print(data[:5])

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


In [13]:
class CBOW(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(2 * context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_porbs = F.log_softmax(out, dim=1)
        return log_porbs

In [15]:
# create the model and train
EMBEDDING_DIM = 100
CONTEXT_SIZE = 2
model = CBOW(len(word_to_ix), EMBEDDING_DIM, CONTEXT_SIZE)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(10):
    print('epoch {}'.format(epoch))
    print('*' * 10)
    total_loss = torch.Tensor([0])
    
    for context, target in data:
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        model.zero_grad()
        log_probs = model(context_idxs)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    print('loss: ', total_loss.item())

epoch 0
**********
loss:  228.74998474121094
epoch 1
**********
loss:  183.2062225341797
epoch 2
**********
loss:  141.5687713623047
epoch 3
**********
loss:  101.90205383300781
epoch 4
**********
loss:  67.41657257080078
epoch 5
**********
loss:  40.27521896362305
epoch 6
**********
loss:  23.017635345458984
epoch 7
**********
loss:  13.898042678833008
epoch 8
**********
loss:  9.23575210571289
epoch 9
**********
loss:  6.70901346206665


In [9]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

make_context_vector(data[0][0], word_to_ix)

tensor([  4,  11,   5,  37])